In [2]:
import requests
from bs4 import BeautifulSoup
import time
import json
from collections import defaultdict

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

/Users/user/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

driver = webdriver.Safari()

In [59]:
job_category_dict = {
	10110: "소프트웨어 엔지니어",
	873: "웹 개발자",
	872: "서버 개발자",
	669: "프론트엔드 개발자",
	660: "자바 개발자",
	900: "C,C++ 개발자",
	899: "파이썬 개발자",
	1634: "머신러닝 엔지니어",
	674: "DevOps / 시스템 관리자",
	665: "시스템,네트워크 관리자",
	655: "데이터 엔지니어",
	895: "Node.js 개발자",
	677: "안드로이드 개발자",
	678: "iOS 개발자",
	658: "임베디드 개발자",
	877: "개발 매니저",
	1024: "데이터 사이언티스트",
	1026: "기술지원",
	676: "QA,테스트 엔지니어",
	672: "하드웨어 엔지니어",
	1025: "빅데이터 엔지니어",
	671: "보안 엔지니어",
	876: "프로덕트 매니저",
	10111: "크로스플랫폼 앱 개발자",
	1027: "블록체인 플랫폼 엔지니어",
	10231: "DBA",
	893: "PHP 개발자",
	661: ".NET 개발자",
	896: "영상,음성 엔지니어",
	10230: "ERP전문가",
	939: "웹 퍼블리셔",
	898: "그래픽스 엔지니어",
	795: "CTO,Chief Technology Officer",
	10112: "VR 엔지니어",
	1022: "BI 엔지니어",
	894: "루비온레일즈 개발자",
	793: "CIO,Chief Information Officer"
}

채용공고 url 추출

In [80]:
job_category_url = {}

In [83]:
for key in job_category_dict:
	print(key)
	driver.get(f"https://www.wanted.co.kr/wdlist/518/{key}")
	
	_page_source = ""
	while True:
		driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(1)
		if _page_source == driver.page_source:
			break
		else:
			_page_source = driver.page_source
	
	soup = BeautifulSoup(_page_source, 'html.parser')
	ul_element = soup.find('ul', {'data-cy': 'job-list'})
	job_category_url[key] = [a_tag['href'] for a_tag in ul_element.find_all('a') if a_tag.get('href', '').startswith('/wd/')]
	
	with open('../data/wanted.url.json', 'w') as f:
		json.dump(job_category_url, f)

10110
873
872
669
900
660
899
1634
674
665
655
895
677
678
658
877
1024
1026
676
672
1025
671
876
10111
1027
10231
893
661
896
10230
939
898
795
10112
1022
894
793


상세정보 크롤링

In [3]:
with open("../data/wanted.url.json") as f:
	job_category_url = json.load(f)

In [4]:
content_info_dict = defaultdict(dict)
with open("../data/wanted.content.json") as f:
	content_info_dict = json.load(f)

In [5]:
done_list = []
for k,v in content_info_dict.items():
	done_list.extend([
		vv for vv in v
	])

driver = webdriver.Safari()

for key, value in job_category_url.items():
	for v in value:
		if v in done_list: continue
		driver.get(f"https://www.wanted.co.kr{v}")
		time.sleep(1)
		if key not in content_info_dict:
			content_info_dict[key] = dict()
		content_info_dict[key][v] = driver.page_source

	with open('../data/wanted.content.json', 'w') as f:
		json.dump(content_info_dict, f)

KeyboardInterrupt: 

In [ ]:
with open('../data/wanted.content.json', 'w') as f:
	json.dump(content_info_dict, f)

In [41]:
driver.get("https://www.wanted.co.kr/wd/195703")

In [42]:
source = driver.page_source

### 상세정보 추출

- url : 채용공고 url
- title : 채용공고명
- job_category : 직무 분류 코드
- job_name : 직무명
- company_id : 회사 분류 id
- company_name : 회사명
- deadline : 마감일
- tech_list : 기술스택 목록
- academic_background : 학력
- career : 경력 (몇년차 요구하는지)
- location : 위치
- tag_id : 복지 관련 id
- tag_name : 복지 목록

In [2]:
with open("../data/wanted.content.json") as f:
	content_info_dict = json.load(f)

KeyboardInterrupt: 

In [7]:
for key, value in content_info_dict.items():
	for url, page_source in value.items():
		break

In [51]:
driver.get("https://www.wanted.co.kr/wd/195530")

In [83]:
page_source = driver.page_source

In [9]:
url

'/wd/195530'

In [84]:
soup = BeautifulSoup(page_source, 'html.parser')

In [ ]:
TAG2FIELD_MAP = {
	"설명": "description",
	"주요업무": "main_work",
	"자격요건": "qualification",
	"우대사항": "preferences",
	"혜택 및 복지": "welfare",
	"기술스택 ・ 툴": "tech_list"
} 

In [133]:
result = {}

job_header = soup.find("section", class_="JobHeader_className__HttDA")

result['title'] = job_header.find("h2").text

_company_info = job_header.find("span", class_="JobHeader_companyNameText__uuJyu")
result['company_name'] = _company_info.text
result['company_id'] = _company_info.find("a")["href"]

_tag_list = job_header.find("div", class_="Tags_tagsClass__mvehZ").find_all("a")
result['tag_name'] = [tag.text.lstrip("#") for tag in _tag_list]
result['tag_id'] = [tag["href"] for tag in _tag_list]

In [134]:
job_body = soup.find("section", class_="JobDescription_JobDescription__VWfcb")

p_tags = job_body.find_all("p")
h3_tags = job_body.find_all("h3")

for i, p_tag in enumerate(p_tags):
	h3_tag = h3_tags[i-1].text if i != 0 else "설명"
	if h3_tag not in TAG2FIELD_MAP:
		continue
	
	field_name = TAG2FIELD_MAP[h3_tag]
	if field_name == "tech_list":
		result[field_name] = [
			skill.text for skill in p_tag.find("div").find_all("div")
		]
		continue

	result[field_name] = [
		line for lines in p_tag.text.split("<br>") for line in lines.split("• ") if line
	]

In [135]:
result

{'title': '데브옵스 엔지니어 (보안)',
 'company_name': '트리플콤마(골드스푼)',
 'company_id': '/company/8040',
 'tag_name': ['연봉업계평균이상',
  '인원급성장',
  '50명이하',
  '설립4~9년',
  '유연근무',
  '반려동물',
  '맥주',
  '커피',
  'IT, 컨텐츠'],
 'tag_id': ['/tag_search?tag=%25EC%2597%25B0%25EB%25B4%2589%25EC%2597%2585%25EA%25B3%2584%25ED%258F%2589%25EA%25B7%25A0%25EC%259D%25B4%25EC%2583%2581',
  '/tag_search?tag=%25EC%259D%25B8%25EC%259B%2590%25EA%25B8%2589%25EC%2584%25B1%25EC%259E%25A5',
  '/tag_search?tag=50%25EB%25AA%2585%25EC%259D%25B4%25ED%2595%2598',
  '/tag_search?tag=%25EC%2584%25A4%25EB%25A6%25BD4~9%25EB%2585%2584',
  '/tag_search?tag=%25EC%259C%25A0%25EC%2597%25B0%25EA%25B7%25BC%25EB%25AC%25B4',
  '/tag_search?tag=%25EB%25B0%2598%25EB%25A0%25A4%25EB%258F%2599%25EB%25AC%25BC',
  '/tag_search?tag=%25EB%25A7%25A5%25EC%25A3%25BC',
  '/tag_search?tag=%25EC%25BB%25A4%25ED%2594%25BC',
  '/tag_search?tag=IT%252C%2520%25EC%25BB%25A8%25ED%2585%2590%25EC%25B8%25A0'],
 'description': ['트리플콤마는 국내 최초 서류인증 기반 프리미엄 매칭 서비스인 ‘골드스푼’ 을 개

In [89]:
page_source[200000:220000]

'g","title":"보일러 엔지니어","counts":{"job":0}},{"id":808,"image":"https://static.wanted.co.kr/images/tags/458f3018.jpg","title":"세라믹 엔지니어","counts":{"job":0}},{"id":820,"image":"https://static.wanted.co.kr/images/tags/76b1c7be.jpg","title":"시추 엔지니어","counts":{"job":0}},{"id":830,"image":"https://static.wanted.co.kr/images/tags/e855c16c.jpg","title":"소방안전 기술자","counts":{"job":0}},{"id":832,"image":"https://static.wanted.co.kr/images/tags/4c2479b8.jpg","title":"지리학자","counts":{"job":0}},{"id":833,"image":"https://static.wanted.co.kr/images/tags/227883f2.jpg","title":"지질공학자","counts":{"job":0}},{"id":834,"image":null,"title":"보건안전 엔지니어","counts":{"job":0}},{"id":835,"image":null,"title":"고압설계 엔지니어","counts":{"job":0}},{"id":838,"image":"https://static.wanted.co.kr/images/tags/75370c4e-4283-11e6-adae-029e8c03f357.jpg","title":"I\\u0026C 엔지니어","counts":{"job":0}},{"id":840,"image":null,"title":"생산공학 엔지니어","counts":{"job":0}},{"id":841,"image":null,"title":"해양공학자","counts":{"job":0}},{"id":844,"

In [85]:
job_tail = soup.find_all("section", class_="JobWorkPlace_className__ra6rp")

In [86]:
job_tail

[]

In [74]:
job_tail.find_all("div")

AttributeError: 'NoneType' object has no attribute 'find_all'

In [68]:
company_id

'/company/8040'

In [57]:
soup.find_all("div", class_="JobDescription_JobDescription_skill_wrapper__9EdFE")

[<div class="JobDescription_JobDescription_skill_wrapper__9EdFE"><div class="SkillItem_SkillItem__E2WtM">Android</div><div class="SkillItem_SkillItem__E2WtM">Firebase</div><div class="SkillItem_SkillItem__E2WtM">iOS</div><div class="SkillItem_SkillItem__E2WtM">React</div><div class="SkillItem_SkillItem__E2WtM">Java</div><div class="SkillItem_SkillItem__E2WtM">Kotlin</div><div class="SkillItem_SkillItem__E2WtM">Swift</div><div class="SkillItem_SkillItem__E2WtM">TypeScript</div><div class="SkillItem_SkillItem__E2WtM">AWS</div><div class="SkillItem_SkillItem__E2WtM">NodeJS</div><div class="SkillItem_SkillItem__E2WtM">iOS 개발</div><div class="SkillItem_SkillItem__E2WtM">React Native</div><div class="SkillItem_SkillItem__E2WtM">Flutter</div></div>]

In [18]:
'데브옵스 엔지니어' in page_source

True

In [53]:
page_source.find('Firebase')

89561

In [54]:
page_source[88000:90000]

'하고 있으며 이에 채용공고를 올리게 되었습니다.<br><br>• 2023년 10월에는 40대를 위한 신규 서비스인 \'더멤버스\'를 출시하였고 영업을 시작하였습니다.<br><br>* 탁월한 인재에게 최고의 보상과 성취감을 제공하는 트리플콤마에서 재능을 펼쳐보세요! *</span></p><h3>주요업무</h3><p><span>• AWS 인프라 운영 및 비용/성능 최적화 (예: EC2, RDS, .. )<br>• AWS 보안 아키텍처 설계, 구축, 운영 및 자동화 개발<br>• 클라우드 표준 보안 요구 사항 개발<br>• 인프라/어플리케이션 모니터링 시스템 구축 및 운영<br>• Automated CI/CD 시스템 구축 및 운영<br>• 시스템 / 데이터 백업 정책 설정 및 운영</span></p><h3>자격요건</h3><p><span>• AWS 클라우드 환경에서 3년 이상의 서비스 구축 및 운영 경험<br>• VPC flow, GuardDuty, CloudTrail의 로그 분석 경험<br>• 클라우드 환경에서의 보안 서비스 (Security Group, Network ACL, WAF) 정책 설정 경험<br>• ISMS / ISMS-P 인증 심사 대응을 위한 기술 지원 업무 경험이 있는 분</span></p><h3>우대사항</h3><p><span>• ISMS-P 인증 심사원 자격 보유자<br>• AWS 클라우드 환경에서의 보안을 경험해 보신 분<br>• IaC(Infrastructure as Code)에 대한 이해가 있으신 분<br>• SIEM 시스템을 구축 운영해 보신 분<br>• Prowler 보안툴 사용 경험이 있으신 분</span></p><h3>혜택 및 복지</h3><p><span>1. 합당한 보상<br><br>• 뛰어난 인재에게 합당한 보상을 지급합니다. <br>• 열심히 하는만큼 더 받을 수 있도록 성과에 따른 인센티브도 존재합니다.<br><br>2. 근무 여건의 강점<br><br>• 유연근무제를 시행하고 있습니다.<br>- 주 1회 재택근무<br>-

In [36]:
data = json.loads(soup.find("script", id="__NEXT_DATA__").text)

In [46]:
data['props']['pageProps']

{'head': {'195530': {'status': 'active',
   'title_thumb_img': 'https://static.wanted.co.kr/images/company/8040/5wemaaq53ybzu6dt__400_400.jpg',
   'is_newbie': False,
   'currency': 'KRW',
   'logo_thumb_img': 'https://static.wanted.co.kr/images/wdes/0_5.6120bdd5.jpg',
   'logo_img': 'https://static.wanted.co.kr/images/wdes/0_4.6120bdd5.jpg',
   'address': {'country': '한국', 'id': 6357, 'location': '서울'},
   'confirm_time': '2023-12-13T09:43:38',
   'id': 195530,
   'category': 'IT',
   'main_tasks': '• AWS 인프라 운영 및 비용/성능 최적화 (예: EC2, RDS, .. )\n• AWS 보안 아키텍처 설계, 구축, 운영 및 자동화 개발\n• 클라우드 표준 보안 요구 사항 개발\n• 인프라/어플리케이션 모니터링 시스템 구축 및 운영\n• Automated CI/CD 시스템 구축 및 운영\n• 시스템 / 데이터 백업 정책 설정 및 운영',
   'requirements': '• AWS 클라우드 환경에서 3년 이상의 서비스 구축 및 운영 경험\n• VPC flow, GuardDuty, CloudTrail의 로그 분석 경험\n• 클라우드 환경에서의 보안 서비스 (Security Group, Network ACL, WAF) 정책 설정 경험\n• ISMS / ISMS-P 인증 심사 대응을 위한 기술 지원 업무 경험이 있는 분',
   'job_country': 'KR',
   'industry': 'IT, 컨텐츠',
   'company_id': 8040,
   'title_

In [28]:
idx = page_source.find('혜택 및 복지')
page_source[idx-10000:idx+1000]

'</stop></radialGradient></defs></svg></a><ul class="Menu_MenuNav__EOf3X"><li class="Menu_MenuItem__wI1yA" data-attribute-id="gnb" data-gnb-kind="jobs"><a href="/wdlist" class="Menu_jobListLink__GYHAI"><span><span class="SvgIcon_SvgIcon__root__8vwon Menu_categoryIcon__cARwU"><svg class="SvgIcon_SvgIcon__root__svg__DKYBi" viewBox="0 0 20 20"><path fill-rule="evenodd" clip-rule="evenodd" d="M2.45837 5C2.45837 4.51675 2.85012 4.125 3.33337 4.125H16.6667C17.15 4.125 17.5417 4.51675 17.5417 5C17.5417 5.48325 17.15 5.875 16.6667 5.875H3.33337C2.85012 5.875 2.45837 5.48325 2.45837 5ZM2.45837 15C2.45837 14.5168 2.85012 14.125 3.33337 14.125H11.6667C12.15 14.125 12.5417 14.5168 12.5417 15C12.5417 15.4832 12.15 15.875 11.6667 15.875H3.33337C2.85012 15.875 2.45837 15.4832 2.45837 15ZM3.33337 9.125C2.85012 9.125 2.45837 9.51675 2.45837 10C2.45837 10.4832 2.85012 10.875 3.33337 10.875H16.6667C17.15 10.875 17.5417 10.4832 17.5417 10C17.5417 9.51675 17.15 9.125 16.6667 9.125H3.33337Z"></path><path d=

In [23]:
soup.find_all("meta")

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>,
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>,
 <meta content="no-store" http-equiv="cache-control"/>,
 <meta content="website" property="og:type"/>,
 <meta content="ko_KR" property="og:locale"/>,
 <meta content="316787678519888" property="fb:app_id"/>,
 <meta content="#fffffe" name="theme-color"/>,
 <meta content="#ffffff" name="msapplication-TileColor"/>,
 <meta content="//static.wanted.co.kr/favicon/new/144x144.png" name="msapplication-TileImage"/>,
 <meta content="ef4f7f8504106401fe915b922514a5b352f59ca2" name="naver-site-verification"/>,
 <meta content="KEXLzgQBOVNqw8uNBWOXYOz2L4vUApbKxqxV6twdkxQ" name="google-site-verification"/>,
 <meta content="[트리플콤마(골드스푼)] 데브옵스 엔지니어 (보안) 채용 | 원티드" property="og:title"/>,
 <meta content="트리플콤마(골드스푼)에 지원하고 합격하면 보상금 50만원
 포지션: 데브옵스 엔지니어 (보안)
 주요 업무: • AWS 인프라 운영 및 비용/성능 최적화 (예: EC2, RDS, .. )
 • AWS 보안 아키텍처 설계, 구축, 운영 및 자동화 개발
 • 클라우드 표준 

---

In [49]:
driver.get("https://www.wanted.co.kr/wdlist/518")

In [50]:
driver.maximize_window()

In [57]:
element = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[2]/nav/ul/li[1]/div/button')

In [58]:
driver.execute_script("arguments[0].setAttribute('aria-expanded', 'true')", element)


In [21]:
element_to_hover = driver.find_element(By.CLASS_NAME, 'Menu_MenuItem__wI1yA')

In [27]:
hover = ActionChains(driver).move_to_element(element_to_hover)
hover.perform()

WebDriverWait(driver, 10).until(
    lambda d: d.find_element(By.CLASS_NAME, 'Menu_MenuExpandableButton__vPEIA').get_attribute("aria-expanded") == "true"
)

TimeoutException: Message: 


TimeoutException: Message: 


In [11]:
soup = BeautifulSoup(driver.page_source, 'html')

In [13]:
'빅데이터 엔지니어' in soup

False

In [8]:
driver.find_element(By.CLASS_NAME, 'Menu_jobListLink__GYHAI')

<selenium.webdriver.remote.webelement.WebElement (session="FFE851B4-400B-4C5B-B2DA-6F020D0243B0", element="node-B29B2C7A-58A2-4AE7-BEA1-913CE9ABEC0A")>

In [34]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

id_list = []
recruit_page = 1
with requests.Session() as s:
	response = s.get(f"https://www.saramin.co.kr/zf_user/search?cat_kewd=83&recruitPageCount=100&recruitPage={recruit_page}", headers=headers)
	
soup = BeautifulSoup(response.text, 'html.parser')
contents = soup.find_all('div', class_='item_recruit')
if not contents:
	break
for content in contents:
	id_list.append(content.get('value'))

recruit_page += 1
with requests.Session() as s:
	response = s.get(f"https://www.saramin.co.kr/zf_user/search?cat_kewd=83&recruitPageCount=100&recruitPage={recruit_page}", headers=headers)

SyntaxError: 'break' outside loop (20197525.py, line 13)

In [50]:
type(content)

bs4.element.Tag

In [60]:
for content in soup.find_all('div', class_='item_recruit'):
	print(content.find('h2', class_='job_tit').find('a').get('title'))
	# print(content)

	break

펜타시스템 37기 신입사원 공개채용


In [19]:
len(id_list)

755

In [23]:
contents = soup.find_all('div', class_='item_recruit')

In [5]:
recruit_content_list = []

for i, _id in enumerate(id_list):
	with requests.Session() as s:
		response_detail = s.get(f"https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx={_id}", headers=headers)
	
	soup_detail = BeautifulSoup(response_detail.text, 'html.parser')

	recruit_content = ""
	for i, content in enumerate(soup_detail.find_all('script')):
		if 'recruit_content' in content.text:
			recruit_content = content
			break
	
	if recruit_content == "":
		continue
	
	content_text = recruit_content.text
	start_index = content_text.index('"recruit_contents":"')
	end_index = content_text.index('","kindness_expired_dt"')
	recruit_contents = content_text[start_index+len('"recruit_contents":"'):end_index]
	
	recruit_content_list.append(
		recruit_contents.encode('utf-8').decode('unicode_escape')
    )
	
	if i >= 10:
		break

/var/folders/fb/xtftd5y96nl3xc10kvb6cd4w0000gn/T/ipykernel_12548/3322587434.py:24: DeprecationWarning: invalid escape sequence '\/'
  recruit_contents.encode('utf-8').decode('unicode_escape')


In [29]:
soup_detail.find('h1', _class="tit_job")

In [6]:
len(recruit_content_list)

1

In [7]:
recruit_content_list

['S-Eden 솔루션사업팀 웹 개발 엔지니어 모집 (신입\\/경력) 2009년 04월 30일에 설립된 시스템 소프트웨어 개발 및 공급업업종의 소프트웨어자문，개발및공급,컴퓨터관련,컴퓨터관련,유지보수용역사업을 하는 중소기업, 주식회사, 병역특례 인증업체기업 입니다.  및  공통  ㆍ 대졸 이상 (2,3년) 에스이든 JAVA SPRING 솔루션 개발자 모집솔루션 사업팀 0명  ㆍ경력 : 신입\\/경력 1년 ~ 10년(주임, 대리, 과장급) 전자정부 프레임워크 사용.  ㆍ정보처리기사 ㆍ해당직무 근무경험, 타 언어(PHP, C 등) 사용 가능자  ㆍ:정규직(수습기간)-3개월 ㆍ:주 5일(월~금) ㆍ:() 서울 금천구 가산동 481-2 에이스하이엔드타워9차 218 - 서울 1호선 가산디지털단지 에서 800m 이내       ㆍ:2023년 9월 13일 (수) 19시 ~ 2023년 10월 13일 (금) 24시 ㆍ: ㆍ: ㆍ:- 이력서, 자기소개서  ㆍ']

In [10]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [15]:
token_list = []
for content in recruit_content_list:
	token_list.extend([
		token.form for token in kiwi.tokenize(content) if token.tag == 'NNG'
	])

In [16]:
token_list

['솔루션',
 '사업',
 '팀',
 '웹',
 '개발',
 '엔지니어',
 '모집',
 '신입',
 '경력',
 '설립',
 '시스템',
 '소프트웨어',
 '개발',
 '공급',
 '업',
 '업종',
 '소프트웨어',
 '자문',
 '개발',
 '공급',
 '컴퓨터',
 '관련',
 '컴퓨터',
 '관련',
 '유지',
 '보수',
 '용역',
 '사업',
 '중소기업',
 '주식회사',
 '병역',
 '특례',
 '인증',
 '업체',
 '기업',
 '공통',
 '대졸',
 '이상',
 '솔루션',
 '개발자',
 '모집',
 '솔루션',
 '사업',
 '팀',
 '경력',
 '신입',
 '경력',
 '주임',
 '대리',
 '과장',
 '급',
 '전자',
 '정부',
 '프레임',
 '워크',
 '사용',
 '정보',
 '처리',
 '기사',
 '해당',
 '직무',
 '근무',
 '경험',
 '언어',
 '사용',
 '가능',
 '정규직',
 '수습',
 '기간',
 '주',
 '월',
 '금',
 '가산',
 '디지털',
 '단지',
 '이내',
 '수',
 '금',
 '이력서',
 '자기소개서']

In [ ]:
from slimit.parser import Parser
from slimit.visitors import nodevisitor
from slimit import ast